CountryandRegionMap__c

Currency_Management2__c

Part__c

Shipment_Order__c

Tax_Condition__c

Tax_Structure_Per_Country__c

In [5]:
import pandas, os, math
pd = pandas

results_df = {}
results_df['CountryandRegionMap__c'] = pd.read_excel('TaxCalcFiles/CountryandRegionMap__c.xlsx',index_col=0)
results_df['CPA_Costing__c'] = pd.read_excel('TaxCalcFiles/CPA_Costing__c.xlsx',index_col=0)
results_df['Currency_Management2__c'] = pd.read_excel('TaxCalcFiles/Currency_Management2__c.xlsx',index_col=0)
results_df['Freight__c'] = pd.read_excel('TaxCalcFiles/Freight__c.xlsx',index_col=0)
results_df['Part__c'] = pd.read_excel('TaxCalcFiles/Part__c.xlsx',index_col=0)
results_df['Shipment_Order__c'] = pd.read_excel('TaxCalcFiles/Shipment_Order__c.xlsx',index_col=0)
results_df['Shipment_Order_Package__c'] = pd.read_excel('TaxCalcFiles/Shipment_Order_Package__c.xlsx',index_col=0)
results_df['Task'] = pd.read_excel('TaxCalcFiles/Task.xlsx',index_col=0)
results_df['Tax_Condition__c'] = pd.read_excel('TaxCalcFiles/Tax_Condition__c.xlsx',index_col=0)
results_df['Tax_Structure_Per_Country__c'] = pd.read_excel('TaxCalcFiles/Tax_Structure_Per_Country__c.xlsx',index_col=0)

In [6]:
data_dict = results_df.copy()

In [8]:
data_dict.keys()

dict_keys(['CountryandRegionMap__c', 'CPA_Costing__c', 'Currency_Management2__c', 'Freight__c', 'Part__c', 'Shipment_Order__c', 'Shipment_Order_Package__c', 'Task', 'Tax_Condition__c', 'Tax_Structure_Per_Country__c'])

In [29]:
data_dict['Tax_Structure_Per_Country__c']

,Id,Amount__c,Per_Weight__c,Default_Duty_Rate__c,Tax_Type__c,Part_Specific__c,Uses_Total_From_Previous_Step__c,NonCooCondition__c,Max__c,Match_to_CCD__c,...,Additional_Percent__c,Use_Total_From_Order_No__c,Condition_Logic__c,Name,Order_Number__c,Additional_Part_Specific_Tax__c,Country__r.Name,Country__r.CIF_Absolute_value_adjustment__c,Country__r.CIF_Adjustment_Factor__c,Lookup Value
a1c0Y0000034jCFQAY,a1c0Y0000034jCFQAY,NaN,NaN,0.0,Duties,True,False,1,NaN,False,...,NaN,NaN,1 AND 2,Duties,1,NaN,Australia,50,0.5,A1C0Y0000034JCFQAY
a1c0Y0000034jCKQAY,a1c0Y0000034jCKQAY,NaN,NaN,NaN,VAT,False,False,1,NaN,False,...,NaN,NaN,1,VAT,2,NaN,Australia,50,0.5,A1C0Y0000034JCKQAY
a1c0Y0000034jCPQAY,a1c0Y0000034jCPQAY,65.0,NaN,NaN,Other,False,False,2,NaN,False,...,NaN,NaN,1 AND 2,"Processing Fee <10,000 AUS Dollars",3,NaN,Australia,50,0.5,A1C0Y0000034JCPQAY
a1c1v000005C8QxAAK,a1c1v000005C8QxAAK,130.0,NaN,NaN,Other,False,False,1,NaN,False,...,NaN,NaN,1,"Processing Fee >10,000 AUS Dollars",4,NaN,Australia,50,0.5,A1C1V000005C8QXAAK
a1c0Y000002lf1kQAA,a1c0Y000002lf1kQAA,9.0,NaN,NaN,Levies,False,False,1,NaN,False,...,NaN,NaN,1,"01 (< AZN1,000.00)",1,NaN,Azerbaijan,250,5.0,A1C0Y000002LF1KQAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a1c0Y0000034h4xQAA,a1c0Y0000034h4xQAA,NaN,NaN,2.0,Duties,True,False,2,NaN,False,...,NaN,NaN,1 AND 2,Duties,1,NaN,United States,0,0.0,A1C0Y0000034H4XQAA
a1c0Y0000034h52QAA,a1c0Y0000034h52QAA,NaN,NaN,NaN,Levies,False,False,2,538.4,False,...,NaN,NaN,1 AND 2,MPF,2,NaN,United States,0,0.0,A1C0Y0000034H52QAA
a1c1v000005Bt5oAAC,a1c1v000005Bt5oAAC,NaN,NaN,0.0,Other,True,False,2,NaN,False,...,NaN,NaN,1 AND 2 AND 3,China Trade War,3,1.0,United States,0,0.0,A1C1V000005BT5OAAC
a1c1v0000055mC0AAI,a1c1v0000055mC0AAI,NaN,NaN,0.0,VAT,False,False,0,NaN,False,...,NaN,NaN,NaN,VAT,4,NaN,United States,0,0.0,A1C1V0000055MC0AAI


In [9]:
def total_taxes(results_df, lookup_value):
    results_df = results_df.copy()
    so_tax_totals_dict = {}
    li_tax_totals_dict = {}
    
    so_record = get_so_dict(results_df, lookup_value)
    li_records = get_li_dict(results_df, lookup_value)
    clearance_country = so_record['CPA_v2_0__r.Final_Destination__c']
    tax_structure_dict = get_tax_structure_dict(results_df, clearance_country)
    so_has_line_items = so_record['of_Line_Items__c'] > 0
    
    if so_has_line_items:
        for li_record_key in li_records.keys():
            li_record = li_records[li_record_key]
            li_lookup_value = li_records[li_record_key]['Lookup Value']
            li_tax_totals_dict[li_lookup_value] = {}
            
            #print(li_records[li_record_key]['Lookup Value'])
            for tax_record_key in tax_structure_dict.keys():
                tax_record = tax_structure_dict[tax_record_key]
                tax_order_number = str(tax_record['Order_Number__c'])
                tax_type = get_tax_type(tax_record)
                #print(tax_type)
                tax_conditions_dict = get_tax_conditions_dict(results_df, tax_record)
                
                if tax_type == 'Amount':
                    tax_amount = get_tax_amount(tax_record,so_record,tax_type,li_record)
                    #print(tax_amount)
                
                if tax_type != 'Amount':
                    tax_rate = get_tax_rate(tax_record,so_record,tax_type,li_record)
                    taxable_value = get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_record,so_record,tax_type,li_record)
                    #print(tax_rate)
                    #print(taxable_value)
                #print(tax_structure_dict[tax_record_key]['Tax_Type__c'])
                
                if tax_type == 'Amount':
                    tax_total = tax_amount
                else:
                    tax_total = taxable_value * tax_rate
                #print(tax_total)
                
                #if tax_order_number == '12':
                #    print(li_lookup_value)
                #    print(tax_rate)
                #    print(taxable_value)
                #    print(tax_total)
                
                li_tax_totals_dict[li_lookup_value][tax_order_number] = tax_total
                #print(li_tax_totals_dict)
        so_tax_totals_dict = so_tax_totals_rollup(li_tax_totals_dict,so_tax_totals_dict)
        
        
    
    
    else:
        #print(so_record['Lookup Value'])
        for tax_record_key in tax_structure_dict.keys():
            tax_record = tax_structure_dict[tax_record_key]
            tax_order_number = str(tax_record['Order_Number__c'])
            tax_type = get_tax_type(tax_record)
            #print(tax_type)
            tax_conditions_dict = get_tax_conditions_dict(results_df, tax_record)
            
            if tax_type == 'Amount':
                tax_amount = get_tax_amount(tax_record,so_record,tax_type)
                #print(tax_amount)
            
            if tax_type != 'Amount':
                tax_rate = get_tax_rate(tax_record,so_record,tax_type)
                taxable_value = get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_record,so_record,tax_type)
                #print(tax_rate)
                #print(taxable_value)
            #print(tax_structure_dict[tax_record_key]['Tax_Type__c'])
            
            if tax_type == 'Amount':
                tax_total = tax_amount
            else:
                tax_total = taxable_value * tax_rate
                
            #print(tax_total)
            so_tax_totals_dict[tax_order_number] = tax_total
            #print(so_tax_totals_dict)
    
    for tax_record_key in tax_structure_dict.keys():
        tax_record = tax_structure_dict[tax_record_key]
        tax_order_number = str(tax_record['Order_Number__c'])
        tax_max = tax_record['Max__c']
        tax_min = tax_record['Min__c']
        so_tax_totals_dict = apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number)
    
    print(so_tax_totals_dict)

In [10]:
def get_tax_structure_dict(results_df, clearance_country):
    tax_structures = results_df['Tax_Structure_Per_Country__c']
    return tax_structures[tax_structures['Country__r.Name'] == clearance_country].T.to_dict()

In [11]:
def get_so_dict(results_df, so_lookup_value):
    so_df = results_df['Shipment_Order__c']
    so_dict = so_df[so_df['Lookup Value'] == so_lookup_value].T.to_dict()
    so_id = list(so_dict.keys())[0]
    return so_dict[so_id]

In [12]:
def get_li_dict(results_df, so_lookup_value):
    li_df = results_df['Part__c']
    return li_df[li_df['Shipment_Order__r.Client_Reference__c'] == so_lookup_value].T.to_dict()

In [31]:
x = get_li_dict(results_df, 'SO05')
x.keys()

dict_keys(['a0N07000000og8iEAA', 'a0N07000000og8hEAA', 'a0N07000000og8gEAA', 'a0N07000000og8fEAA', 'a0N07000000og8jEAA', 'a0N07000000og8dEAA', 'a0N07000000og8cEAA', 'a0N07000000og8bEAA', 'a0N07000000og8eEAA'])

In [13]:
def get_tax_type(tax_structure_item):
    tax_type = tax_structure_item['Tax_Type__c']
    tax_rate = tax_structure_item['Rate__c']
    tax_amount = tax_structure_item['Amount__c']
    tax_part_specific = tax_structure_item['Part_Specific__c']
    if tax_type == 'VAT' and tax_part_specific == True:
        return 'VAT'
    elif tax_type == 'Duties' and tax_part_specific == True:
        return 'Duties Part Specific'
    elif tax_type != 'Duties' and tax_part_specific == True:
        return 'Other Part Specific'
    elif pd.isna(tax_rate):
        return 'Amount'
    else:
        return 'Rate Non Part Specific'        

In [14]:
def get_tax_conditions_dict(results_df, tax_structure_item):
    tax_calculation_id = tax_structure_item['Id']
    tax_conditions = results_df['Tax_Condition__c']
    return tax_conditions[tax_conditions['Tax_Structure_Per_Country__c'] == tax_calculation_id].T.to_dict()

In [15]:
def get_tax_amount(tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    so_tax_amount = tax_structure_item['Amount__c']
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    number_of_line_items = so_tax_item['of_Line_Items__c']
    shipment_value = so_tax_item['Shipment_Value_USD__c']
    
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c']
        li_proportion = 1 / number_of_line_items
        #print(li_proportion)
        tax_amount = so_tax_amount * li_proportion
        return tax_amount
    else:
        return so_tax_amount

In [16]:
def get_tax_rate(tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    tax_structure_item_rate = tax_structure_item['Rate__c']/100
    tax_structure_item_default_rate = tax_structure_item['Default_Duty_Rate__c']/100
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    
    if tax_type == 'VAT':
        if so_has_line_items:
            if pd.isna(li_tax_item['Specific_VAT_Rate__c']):
                tax_item_rate = tax_structure_item_rate
            else:
                tax_item_rate = li_tax_item['Specific_VAT_Rate__c']/100
        else:
            tax_item_rate = tax_structure_item_rate
    
    elif tax_type == 'Duties Part Specific':
        if so_has_line_items:
            if pd.isna(li_tax_item['Rate__c']):
                tax_item_rate = tax_structure_item_default_rate
            else:
                tax_item_rate = li_tax_item['Rate__c']/100
        else:
            tax_item_rate = tax_structure_item_default_rate
    
    elif tax_type == 'Other Part Specific':
        if so_has_line_items:
            if pd.isna(li_tax_item['Part_Specific_Rate_1__c']):
                tax_item_additional_rate_1 = 0
            else:
                tax_item_additional_rate_1 = li_tax_item['Part_Specific_Rate_1__c']/100
            if pd.isna(li_tax_item['Part_Specific_Rate_2__c']):
                tax_item_additional_rate_2 = 0
            else:
                tax_item_additional_rate_2 = li_tax_item['Part_Specific_Rate_2__c']/100
            if pd.isna(li_tax_item['Part_Specific_Rate_3__c']):
                tax_item_additional_rate_3 = 0
            else:
                tax_item_additional_rate_3 = li_tax_item['Part_Specific_Rate_3__c']/100
            tax_structure_item_additional = int(tax_structure_item['Additional_Part_Specific_Tax__c'])
            tax_item_rate = eval(f'tax_item_additional_rate_{tax_structure_item_additional}')
        else:
            tax_item_rate = 0
    
    elif tax_type == 'Rate Non Part Specific':
        tax_item_rate = tax_structure_item_rate        
          
    return tax_item_rate

In [17]:
def get_cif_value(tax_structure_item,so_tax_item,li_tax_item={}):
    cif_absolute_value_adjustment = tax_structure_item['Country__r.CIF_Absolute_value_adjustment__c']
    cif_adjustment_factor = tax_structure_item['Country__r.CIF_Adjustment_Factor__c']/100
    
    cpa_cif = so_tax_item['CPA_v2_0__r.CIF_Freight_and_Insurance__c']
    client_actual_freight = so_tax_item['Client_Actual_Freight_Value__c']
    client_actual_insurance = so_tax_item['Client_Actual_Insurance_Value__c']
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    shipment_value = so_tax_item['Shipment_Value_USD__c']
    
    if pd.isna(cpa_cif):
        so_cif_value = (shipment_value + cif_absolute_value_adjustment) * (1 + cif_adjustment_factor)
    else:
        so_cif_value = shipment_value + client_actual_freight + client_actual_insurance
        
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c']
        li_proportion = li_value / shipment_value
        li_cif_value = so_cif_value * li_proportion
        return li_cif_value
    else:
        return so_cif_value

In [18]:
def get_fob_value(tax_structure_item,so_tax_item,li_tax_item={}):
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    so_fob_value = so_tax_item['Shipment_Value_USD__c']
    
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c']
        li_proportion = li_value / so_fob_value
        li_fob_value = so_fob_value * li_proportion
        return li_fob_value
    else:
        return so_fob_value

In [19]:
def get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,order_number,lookup_value):
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    
    if so_has_line_items:
        return li_tax_totals_dict[lookup_value][order_number]
    else:
        return so_tax_totals_dict[order_number]

In [20]:
def get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    applied_to_string = tax_structure_item['Applied_to_Value__c']
    order_number = tax_structure_item['Order_Number__c']
    additional_percent = tax_structure_item['Additional_Percent__c']
    additional_applies_to = tax_structure_item['Applies_to_Order__c']
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    so_lookup_value = so_tax_item['Lookup Value']
    
    if (tax_type == 'VAT' or tax_type == 'Duties Part Specific' or tax_type == 'Other Part Specific' or tax_type == 'Rate Non Part Specific'):
        if so_has_line_items:
            li_lookup_value = li_tax_item['Lookup Value']
            if applied_to_string == 'CIF':
                taxable_value = get_cif_value(tax_structure_item, so_tax_item, li_tax_item)
            elif applied_to_string == 'FOB':
                taxable_value = get_fob_value(tax_structure_item, so_tax_item, li_tax_item)
            else:
                taxable_value = get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,applied_to_string,li_lookup_value)
            
        else:
            if applied_to_string == 'CIF':
                taxable_value = get_cif_value(tax_structure_item, so_tax_item)
            elif applied_to_string == 'FOB':
                taxable_value = get_fob_value(tax_structure_item, so_tax_item)
            else:
                taxable_value = get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,applied_to_string,so_lookup_value)
        
        if not pd.isna(additional_percent):
            taxable_value = taxable_value * (1 + additional_percent/100)
        
        if not pd.isna(additional_applies_to):
            for additional_applies_to_item in additional_applies_to.split(','):
                if so_has_line_items:
                    taxable_value += get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,additional_applies_to_item,li_lookup_value)
                else:
                    taxable_value += get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,additional_applies_to_item,so_lookup_value)
                    
    return taxable_value

In [21]:
def so_tax_totals_rollup(li_tax_totals_dict,so_tax_totals_dict):
    for tax_order in li_tax_totals_dict[list(li_tax_totals_dict.keys())[0]].keys():
        so_tax_totals_dict[tax_order] = 0
    
    for li in li_tax_totals_dict.keys():
        for tax_order in li_tax_totals_dict[li].keys():
            so_tax_totals_dict[tax_order] += li_tax_totals_dict[li][tax_order]
                    
    return so_tax_totals_dict

In [22]:
def apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number):
    tax_total = so_tax_totals_dict[tax_order_number]
    
    if not pd.isna(tax_max):
        tax_total = min(tax_total,tax_max)
   
    if not pd.isna(tax_min):
        tax_total = max(tax_total,tax_min)

    so_tax_totals_dict[tax_order_number] = tax_total
    
    return so_tax_totals_dict

In [130]:
total_taxes(results_df, 'SO05')

{'1': 0.0, '2': 27090.0, '3': 0.0}


In [ ]:
CountryandRegionMap__c
Currency_Management2__c
Part__c
Shipment_Order__c
Tax_Condition__c
Tax_Structure_Per_Country__c